# Deploying Machine Learning Models using ksonnet and Ambassador


## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - [Minikube](https://github.com/kubernetes/minikube) version v0.24.0 or greater
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [ksonnet client](https://ksonnet.io/)

Start minikube and ensure custom resource validation is activated and there is 5G of memory. 

**2018-06-13** : At present we find the most stable version of minikube across platforms is 0.25.2 as there are issues with 0.26 and 0.27 on some systems. We also find the default VirtualBox driver can be problematic on some systems so we suggest using the [KVM2 driver](https://github.com/kubernetes/minikube/blob/master/docs/drivers.md#kvm2-driver).

Your start command would then look like:
```
minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC
```

## Setup

When you have a running minikube cluster run:


In [1]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding "kube-system-cluster-admin" created


In [2]:
!kubectl create namespace seldon

namespace "seldon" created


In [7]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


## Install Seldon Core
Create a ksonnet app and install the prototypes from our registry.

In [3]:
!ks init my-ml-deployment 

INFO Using context "minikube" from kubeconfig file "/home/clive/.kube/config" 
INFO Creating environment "default" with namespace "default", pointing to "version:v1.9.4" cluster at address "https://192.168.39.165:8443" 
INFO Generating ksonnet-lib data at path '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/lib/ksonnet-lib/v1.9.4' 


In [4]:
!cd my-ml-deployment && \
    ks registry add seldon-core ../../seldon-core && \
    ks pkg install seldon-core/seldon-core@master && \
    ks generate seldon-core seldon-core --withApife=false --withAmbassador=true --namespace=seldon --withRbac=true

INFO Retrieved 7 files                            
INFO Writing component at '/home/clive/work/seldon-core/fork-seldon-core/notebooks/my-ml-deployment/components/seldon-core.jsonnet' 


In [5]:
!cd my-ml-deployment && \
      ks apply default

INFO Applying services seldon.seldon-core-ambassador 
INFO Creating non-existent services seldon.seldon-core-ambassador 
INFO Applying clusterroles seldon-crd             
INFO Creating non-existent clusterroles seldon-crd 
INFO Applying roles seldon.seldon-local           
INFO Creating non-existent roles seldon.seldon-local 
INFO Applying rolebindings seldon.seldon          
INFO Creating non-existent rolebindings seldon.seldon 
INFO Applying clusterrolebindings seldon.seldon   
INFO Creating non-existent clusterrolebindings seldon.seldon 
INFO Applying rolebindings seldon.ambassador      
INFO Creating non-existent rolebindings seldon.ambassador 
INFO Applying serviceaccounts seldon.seldon       
INFO Creating non-existent serviceaccounts seldon.seldon 
INFO Applying services seldon.seldon-core-redis   
INFO Creating non-existent services seldon.seldon-core-redis 
INFO Applying customresourcedefinitions seldondeployments.machinelearning.seldon.io 
INFO Creating non-existent customre

In [9]:
!kubectl rollout status deploy/seldon-core-seldon-cluster-manager
!kubectl rollout status deploy/seldon-core-ambassador

deployment "seldon-core-seldon-cluster-manager" successfully rolled out
deployment "seldon-core-ambassador" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador**:

```
kubectl port-forward $(kubectl get pods -n seldon -l service=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

Install gRPC modules for the prediction protos.

In [10]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

Illustration of both REST and gRPC requests. 

## Create Seldon Deployment

**Check everything is running before continuing**

In [11]:
!kubectl apply -f resources/model.json -n seldon

seldondeployment "seldon-model" created


Check status of deployment before continuing. **ReplicasAvailable must be equal to 1**  First time might take some time to download images.

In [12]:
!kubectl get seldondeployments seldon-model -o jsonpath='{.status}' -n seldon

map[predictorStatus:[map[replicasAvailable:1 name:test-deployment-example-classifier-0 replicas:1] map[name:test-deployment-example-svc-orch replicas:1 replicasAvailable:1]]]

## Get predictions

In [13]:
from seldon_utils import *
API_AMBASSADOR="localhost:8003"

#### REST Request

In [14]:
rest_request_ambassador("seldon-model",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "924vm0l4s9rq4qcdcmvj2c491v",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [1, 1],
      "values": [0.09309506222685876]
    }
  }
}


#### gRPC Request

In [15]:
 grpc_request_ambassador("seldon-model",API_AMBASSADOR)

meta {
  puid: "m445sra0c0djcvsmrscirov6ek"
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.09935408521916099
  }
}



## Adding Authentication
We will add the example authentication from the Ambassador tutorial.

In [3]:
!kubectl apply -f resources/ambassador-auth-service-setup.yaml -n seldon

service "example-auth" created
deployment "example-auth" created


** Need to wait until running before adding Ambassador config **

In [5]:
!kubectl get pods -n seldon

NAME                                                    READY     STATUS    RESTARTS   AGE
example-auth-54df78c744-qz9s4                           1/1       Running   0          15s
seldon-core-ambassador-7fb4575f6b-5h9xc                 2/2       Running   0          1h
seldon-core-redis-54f855dd66-5vs4v                      1/1       Running   0          1h
seldon-core-seldon-cluster-manager-79f569d785-9nj4h     1/1       Running   0          1h
test-deployment-example-classifier-0-678f5694bd-m9njb   1/1       Running   0          1h
test-deployment-example-svc-orch-5fcf864665-l2w62       1/1       Running   0          1h


In [6]:
!kubectl apply -f resources/ambassador-auth-service-config.yaml -n seldon

service "example-auth" configured


Show failed request when auth is running

In [2]:
rest_request_ambassador("seldon-model",API_AMBASSADOR)

401



Show successful request with auth

In [3]:
rest_request_ambassador_auth("seldon-model","username","password",API_AMBASSADOR)

200
{
  "meta": {
    "puid": "v0htigtcfdmemoh9ub42bse091",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["proba"],
    "tensor": {
      "shape": [2, 1],
      "values": [0.09131694949001515, 0.078402214699956]
    }
  }
}


# Tear down

In [ ]:
!kubectl delete -f resources/ambassador-auth-service-setup.yaml -n seldon

In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
!cd my-ml-deployment && ks delete default

In [ ]:
!rm -rf my-ml-deployment